In [ ]:
import numpy as np
import pickle
from pathlib import Path
import bw2calc as bc
import bw2data as bd
from gwp_uncertainties import add_bw_method_with_gwp_uncertainties
import plotly.graph_objects as go
import stats_arrays as sa
from plotly.subplots import make_subplots

if __name__ == "__main__":

    time_horizon = 100 # TODO choose

    project = "GSA for protocol"
    bd.projects.set_current(project)

    add_bw_method_with_gwp_uncertainties(time_horizon, verbose=True)

    co = bd.Database('CH consumption 1.0')
    demand_act = co.search('food sector')[0]
    demand = {demand_act: 1}
    print(demand_act)
    method = ('IPCC 2013', 'climate change', 'GWP {}a'.format(time_horizon))
    uncertain_method = method + ('uncertain',)

    lca = bc.LCA(demand, method)
    lca.lci()
    lca.lcia()
    print("{} LCA score with standard IPCC method".format(lca.score))
    ulca = bc.LCA(demand, uncertain_method)
    ulca.lci()
    ulca.lcia()
    print("{} LCA score with uncertain IPCC method".format(ulca.score))

    

In [ ]:
iterations = 2000
write_dir = Path("write_files")
path_lca_scores_certain = write_dir / "lca_scores_certain_gwp_{}.pickle".format(iterations)
path_lca_scores_uncertain = write_dir / "lca_scores_uncertain_gwp_{}.pickle".format(iterations)

In [ ]:
%%time

if path_lca_scores_certain.exists():
    with open(path_lca_scores_certain, 'rb') as f:
        lca_scores_certain_gwp = pickle.load(f)
else:
    mc_certain_gwp = bc.ParallelMonteCarlo(demand, method, iterations)
    lca_scores_certain_gwp = np.array(mc_certain_gwp.calculate())
    with open(path_lca_scores_certain, 'wb') as f:
        pickle.dump(lca_scores_certain_gwp, f)

if path_lca_scores_uncertain.exists():
    with open(path_lca_scores_uncertain, 'rb') as f:
        lca_scores_uncertain_gwp = pickle.load(f)
else:
    mc_uncertain_gwp = bc.ParallelMonteCarlo(demand, uncertain_method, iterations)
    lca_scores_uncertain_gwp = np.array(mc_uncertain_gwp.calculate())
    with open(path_lca_scores_uncertain, 'wb') as f:
        pickle.dump(lca_scores_uncertain_gwp, f)
    
print("LCA scores W/O  uncertainties in GWP -> std={}".format(
        np.std(lca_scores_certain_gwp))
    )
print("LCA scores WITH uncertainties in GWP -> std={}".format(
    np.std(lca_scores_uncertain_gwp))
)


# Plot LCA scores

In [ ]:
num_bins = 60
bin_min = min(np.hstack([lca_scores_certain_gwp, lca_scores_uncertain_gwp]))
bin_max = max(np.hstack([lca_scores_certain_gwp, lca_scores_uncertain_gwp]))
bins_ = np.linspace(bin_min, bin_max, num_bins, endpoint=True)
freq_certain, bins_certain = np.histogram(lca_scores_certain_gwp, bins=bins_, density=True)
freq_uncertain, bins_uncertain = np.histogram(lca_scores_uncertain_gwp, bins=bins_, density=True)
    
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=bins_certain,
        y=freq_certain,
        name='GWP values W/O uncertainties',
        opacity=0.7,
    ),
)
fig.add_trace(
    go.Bar(
        x=bins_uncertain,
        y=freq_uncertain,
        name='GWP values WITH uncertainties',
        opacity=0.7,
    ),
)
fig.update_layout(barmode="overlay")
fig.update_yaxes(title_text='Frequency')
fig.update_xaxes(title_text='LCA scores, [kg CO2-eq]')
fig.show()
fig.write_html("write_files/lca_scores_gwp_uncertainties.html")

# Plot GWP with uncertainties

In [ ]:
bw_uncertain_method = bd.Method(uncertain_method)

In [ ]:
all_flows = bw_uncertain_method.load()
uncertain_flows = []
inds = []
for i,flow in enumerate(all_flows):
    try:
        flow[1].get('uncertainty type')
        flow_name = bd.get_activity(flow[0])['name']
        if flow_name not in uncertain_flows:
            uncertain_flows.append(flow_name)
            inds.append(i)
    except:
        pass
# unique_uncertain_flows = set(uncertain_flows)
flows_uncertain_list = [all_flows[i] for i in inds]

In [ ]:
data = []
flow_names = []
iterations = 2000
for flow in flows_uncertain_list:
    if flow[1]['uncertainty type'] == sa.NormalUncertainty.id:
        x = np.random.normal(flow[1]['loc'], flow[1]['scale'], iterations)
    elif flow[1]['uncertainty type'] == sa.UniformUncertainty.id:
        x = (flow[1]['maximum'] - flow[1]['minimum'])*np.random.rand(iterations) + flow[1]['minimum']
    act = bd.get_activity(flow[0])
    flow_name = "{} {}".format(act['name'], act['categories'])[:45]
    data.append(
        {
            'name': flow_name,
            'static': flow[1]['amount'],
            'x': x
        }
    )
    flow_names.append(flow_name)

In [ ]:
n_plots = len(flows_uncertain_list)
ncols = 4
nrows = int(np.ceil(n_plots/ncols))
fig = make_subplots(
    rows=nrows,
    cols=ncols,
    shared_yaxes=True,
    subplot_titles=flow_names,
)
i = 0
for row in range(nrows):
    for col in range(ncols):
        if i < n_plots:
            values = data[i]['x']
            freq, bins = np.histogram(values, bins=num_bins)
            fig.add_trace(
                go.Bar(
                    x=bins,
                    y=freq,
                    showlegend=False,
                    marker_color = 'blue',
                    opacity=0.7,
                ),
                row=row+1,
                col=col+1,
            )
            fig.add_trace(
                go.Scatter(
                    x=[data[i]['static']],
                    y=[0],
                    showlegend=False,
                    marker_color = 'red',
                    mode="markers",
                    marker_symbol = 'x',
                    marker_size=15,
                    opacity=0.7,
                ),
                row=row+1,
                col=col+1,
            )
            i += 1 
fig.update_yaxes(title_text="Frequency", col=1)
fig.update_xaxes(title_text="GWP", row=nrows)
fig.update_layout(
    width=400*ncols, height=300*nrows,
)
fig.show()
fig.write_html("write_files/gwp_uncertainties.html")

In [ ]:
fig.write_image("write_files/gwp_uncertainties.svg")